In [2]:
# 교차검증
import pandas as pd
wine = pd.read_csv("https://raw.githubusercontent.com/leekyuyoung4/ml0719/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9DV/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
x = wine[['alcohol','sugar','pH']].to_numpy()
x

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

In [8]:
x = wine.iloc[:,:-1].to_numpy()
y = wine.iloc[:,-1].to_numpy()
x.shape, y.shape

((6497, 3), (6497,))

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2,random_state=42)

In [11]:
help(train_test_split)


Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets.
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to 

In [17]:
x_train.shape, x_test.shape

((5197, 3), (1300, 3))

In [18]:
x_train_sub,x_test_sub,y_train_sub,y_test_sub= train_test_split(x_train,y_train, test_size=0.2,random_state=42)

In [19]:
from sklearn.tree import DecisionTreeClassifier
dt =  DecisionTreeClassifier(random_state=42)
dt.fit(x_train_sub,y_train_sub)
dt.score(x_train_sub,y_train_sub), dt.score(x_test_sub,y_test_sub)

(0.9971133028626413, 0.864423076923077)

In [20]:
# 교차검증
from sklearn.model_selection import cross_validate
score = cross_validate(dt,x_train,y_train)
score

{'fit_time': array([0.00500965, 0.00498867, 0.00598669, 0.00598168, 0.00701046]),
 'score_time': array([0.00099945, 0.00099993, 0.        , 0.00099874, 0.00099707]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [22]:
import numpy as np
np.mean(score['test_score'])

0.855300214703487

In [24]:
from sklearn.model_selection import StratifiedKFold
score = cross_validate(dt,x_train,y_train,cv=StratifiedKFold())
np.mean(score['test_score'])

0.855300214703487

In [25]:
splitter =  StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
score = cross_validate(dt,x_train,y_train,cv=splitter)
np.mean(score['test_score'])

0.8574181117533719

In [27]:
# 최적의 파라메터 값을 찾아주는 머신러닝 그리드서치
#하이퍼 파라메터 튜닝
from sklearn.model_selection import GridSearchCV
help(DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree._classes:

class DecisionTreeClassifier(sklearn.base.ClassifierMixin, BaseDecisionTree)
 |  DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)
 |  
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"gini", "entropy", "log_loss"}, default="gini"
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "log_loss" and "entropy" both for the
 |      Shannon information gain, see :ref:`tree_mathematical_formulation`.
 |  
 |  splitter : {"best", "random"}, default="best"
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to 

In [28]:
params = {
    'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]
}

In [29]:
gs =  GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [35]:
dt = gs.best_estimator_
dt.score(x_train,y_train), dt.score(x_test,y_test)

(0.9615162593804117, 0.8653846153846154)

In [36]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [39]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [40]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
best_index

0

In [42]:
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [43]:
params = {
    'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005],
    'max_depth':range(5,20),
    'min_samples_split':range(2,100,10)    
}

In [44]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
gs.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005],
                         'min_samples_split': range(2, 100, 10)})

In [45]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [46]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

In [51]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([9, 6, 2, 0, 9, 9, 7, 8, 6, 0], dtype=int64)

In [59]:
randint(2,25).rvs(10)

array([ 4, 18,  5, 13, 22, 15, 11, 18, 10, 10], dtype=int64)

In [60]:
params = {
    'min_impurity_decrease':uniform(0.0001,0.001),
    'max_depth':range(5,20),
    'min_samples_split':randint(2,25).rvs(10)  ,
    'min_samples_leaf':randint(2,25).rvs(10)   
}

In [61]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(x_train,y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000297F17C8040>,
                                        'min_samples_leaf': array([ 4,  4, 18, 20, 13,  8, 24, 17, 19, 15], dtype=int64),
                                        'min_samples_split': array([ 8, 15, 23, 10,  4, 24,  3, 11,  7,  8], dtype=int64)},
                   random_state=42)

In [62]:
np.max(gs.cv_results_['mean_test_score'])

0.8687713778041017

In [63]:
gs.best_params_

{'max_depth': 11,
 'min_impurity_decrease': 0.0005197808564462765,
 'min_samples_leaf': 4,
 'min_samples_split': 7}

In [66]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf =  RandomForestClassifier(n_jobs=-1, random_state=42)
scores =  cross_validate(rf,x_train,y_train,return_train_score=True,n_jobs=-1) 

In [68]:
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8905151032797809, 0.9973541965122431)

In [69]:
rf.fit(x_train,y_train)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [73]:
#oob_score : 랜덤셈플에서 밖의 셈플을 사용해서 좀더 성능을 높일때
rf = RandomForestClassifier(oob_score=True,n_jobs=-1, random_state=42)
rf.fit(x_train,y_train)
rf.oob_score_

0.8934000384837406

In [74]:
from sklearn.ensemble import ExtraTreesClassifier
et= ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores =  cross_validate(et,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8887848893166506, 0.9974503966084433)

In [75]:
et.fit(x_train,y_train)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

In [77]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores =  cross_validate(gb,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8720430147331015, 0.8881086892152563)

In [78]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores =  cross_validate(gb,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8780082549788999, 0.9464595437171814)

In [79]:
gb.fit(x_train, y_train)
gb.feature_importances_

array([0.15872278, 0.68011572, 0.16116151])

In [80]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores =  cross_validate(hgb,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8801241948619236, 0.9321723946453317)

In [82]:
!pip install xgboost

     ------------------------------------- 125.4/125.4 MB 10.2 MB/s eta 0:00:00


In [83]:
from xgboost import XGBClassifier

In [84]:
xgb = XGBClassifier(tree_method='hist', random_state=42)

In [85]:
scores =  cross_validate(xgb,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8799326275264677, 0.9555033709953124)

In [86]:
!pip install lightgbm

  Using cached lightgbm-3.3.2-py3-none-win_amd64.whl (1.0 MB)


In [87]:
from lightgbm import LGBMClassifier
lgb =  LGBMClassifier(random_state=42)
scores =  cross_validate(lgb,x_train,y_train,return_train_score=True,n_jobs=-1) 
np.mean(scores['test_score']), np.mean(scores['train_score'])

(0.8801251203079884, 0.935828414851749)

In [ ]:
# 정형데이터중에서 가장 뛰어난 성능.... 앙상블학습... 
# 랜덤포레스트 : 대표적, 부스스트랩 샘플
# 엑스트라 부트 : 결정트리 노드를 랜덤하게 분할
# 그레디언트 부스팅 : 이전 트리의 손실을 보완하는 식으로 결정트리를연속으로 사용
# 히스토그램 기반 그레디언트 부스팅 : 훈련데이터를  256개 의 정수 구간으로 나누어서 빠르고 높은 성을
# XGBoost
# Light GBM
